# Student Contribution

An awesome variation that includes a tool to make a booking! Thank you! -- Ed

# Project - Airline AI Assistant with Booking Tool

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import pandas as pd

In [ ]:
# Initialization

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

## Tools

* Price and Booking

In [ ]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# "required": ["destination_city", "num_tickets", "ticket_class", "mail_address"]

booking_function = {
        "name": "book_ticket",
        "description": "Book a ticket to a destination city.",
        "parameters": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "The city the customer wants to travel to",
                },
                "num_tickets": {
                    "type": "integer",
                    "description": "The number of tickets to book",
                },
                "ticket_class": {
                    "type": "string",
                    "description": "The class of the ticket (e.g., economy, business)",
                },

                "mail_address": {
                    "type": "string",
                    "description": "Mail address to send the ticket to",
                },
            },
            "required": ["destination_city", "num_tickets", "ticket_class", "mail_address"],
            "additionalProperties": False
        }
}

In [ ]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function},
         {"type": "function", "function": booking_function}]

## Creating Application

In [ ]:
# Creating  ID for each booking
# Produces IDs like 'BK-000001', 'BK-000002'

booking_counter = 0

def generate_unique_booking_id():
    global booking_counter
    booking_counter += 1
    return f"BK-{booking_counter:06d}"  


In [ ]:
# Temporary storage for bookings
bookingDB = pd.DataFrame()

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    global bookingDB
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        destination_city = arguments.get('destination_city')
        price = get_ticket_price(destination_city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": destination_city,"price": price}),
            "tool_call_id": message.tool_calls[0].id
        }
        return response
    
    elif tool_call.function.name == "book_ticket":
        arguments = json.loads(tool_call.function.arguments)
        destination_city = arguments.get('destination_city')
        num_tickets = arguments.get('num_tickets')
        ticket_class = arguments.get('ticket_class')
        mail_address = arguments.get('mail_address')
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": destination_city,"num_tickets": num_tickets, "ticket_class": ticket_class}),
            "tool_call_id": message.tool_calls[0].id
        } 
        # Get the ticket price
        price = int(get_ticket_price(destination_city).replace("$", ""))
        total_price = price * num_tickets
        # Generate a unique booking ID
        booked_ID = generate_unique_booking_id()
        # Define the booking data
        data = {
            "booking_id": [booked_ID],
            "mail_address": [mail_address],
            "destination_city": [destination_city],
            "num_tickets": [num_tickets],
            "ticket_class": [ticket_class],
            "total_price": [total_price],
        }
        booking_temp = pd.DataFrame(data)
        bookingDB = bookingDB._append(booking_temp)
        # Can update with real booking system
        bookingDB.to_csv('bookingDB.csv', index=False)
        return response

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": message})
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat).launch()

## Enjoy, Happy Learning Thanks to Ed!